# COGS 108 - EDA Checkpoint

# Names

- Aditya Sriram
- Weston Chester
- Katherine Gao
- Nicole Liu
- Sophia Conti

<a id='research_question'></a>
# Research Question

Is California on track to reach its goal of 100% zero emission sales by 2035?

Hypothesis: Based on previous data, we believe that California is on track to meet its 100% zero emission sales goal by 2035.

# Setup

In [1]:
## YOUR CODE HERE
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pandas as pd

/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/conda/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


# Data Cleaning

Describe your data cleaning steps here.

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Import ZEV Sales as Dataset
zev_sales = pd.read_csv('Data/zev_sales.csv')
zev_sales = zev_sales[(zev_sales['Data Year'] <= 2021) & (zev_sales['Data Year'] >= 2009)]

# ZEV sales as a percentage of cars registered by county in 2021
reg_2021 = pd.read_csv('Data/Vehicles Registered By County-2021.csv', encoding='utf-16')
reg_2021 = reg_2021.dropna().iloc[0:59].drop(columns=['Trailers', 'Motorcycles'])
reg_2021['Total Vehicles'] = reg_2021['Autos'] + reg_2021['Trucks']
reg_2021['Counties'] = reg_2021['Counties'].apply(str.lower)
reg_2021.sort_values(by = 'Counties', inplace = True)

# Motor Vehicle Registrations (total)
motor_total = pd.read_csv('Data/Motor_Vehicle_Registrations.csv')
motor_cal = motor_total[motor_total['state']=='California'] # Specifically evaluate California data
motor_total = motor_total.groupby('year').sum(numeric_only=True)
motor_cal = motor_cal.drop('state', axis=1)
col_list = ['Auto', 'Bus', 'Truck', 'Motorcycle']
motor_total['Total'] = motor_total[col_list].sum(axis=1)
motor_total['Year'] = pd.Series(list(motor_total.index)).values
motor_cal['Total_Cal'] = motor_cal[col_list].sum(axis=1)
motor_cal = motor_cal.rename(columns={"year":"Year"})

# Vehicle Registration by Fuel Type
veh_reg_2016 = pd.read_csv('Data/Vehicle Registration Counts by State_2016.csv')
veh_reg_2017 = pd.read_csv('Data/Vehicle Registration Counts by State_2017.csv')
veh_reg_2018 = pd.read_csv('Data/Vehicle Registration Counts by State_2018.csv')
veh_reg_2019 = pd.read_csv('Data/Vehicle Registration Counts by State_2019.csv')
veh_reg_2020 = pd.read_csv('Data/Vehicle Registration Counts by State_2020.csv')
veh_reg_2021 = pd.read_csv('Data/Vehicle Registration Counts by State_2021.csv')
all_sets = [veh_reg_2016, veh_reg_2017, veh_reg_2018, veh_reg_2019, veh_reg_2020, veh_reg_2021]
year = 2016
for df in all_sets:
    df['Year']=year
    year += 1
registrations = pd.concat(all_sets).reset_index(drop= True)
category_list = ['Electric (EV)','Plug-In Hybrid Electric (PHEV)','Hybrid Electric (HEV)','Biodiesel','Ethanol/Flex (E85)','Compressed Natural Gas (CNG)','Propane','Hydrogen','Methanol','Gasoline','Diesel']
for category in category_list:
    registrations[category] = registrations[category].astype(str)
    registrations[category] = registrations[category].apply(lambda x: x.replace(',', ''))
    registrations[category] = registrations[category].astype(int)
registrations["Total"] = registrations[category_list].sum(axis=1)
registrations['Percentage'] = registrations['Electric (EV)']/registrations["Total"]
cal_registrations = registrations[registrations["State"] == "California"]

# Data Analysis & Results (EDA)

Carry out EDA on your dataset(s); Describe in this section

In [3]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION